# RTE - Forecast energy consumption in French areas


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Load data

In [2]:
train = pd.read_csv("../input/rte-forecast-energy-consumption-in-french-areas/train.csv")
test = pd.read_csv("../input/rte-forecast-energy-consumption-in-french-areas/test.csv")

train.shape, test.shape

((717414, 3), (60870, 3))

# Understanding Data

In [3]:
train.head()

,metropolitan_area_code,date,energy_consumption
0,3,2017-09-11 02:15:00,477.0
1,3,2017-09-11 02:30:00,454.0
2,3,2017-09-11 03:45:00,398.0
3,3,2017-09-11 04:45:00,398.0
4,3,2017-09-11 05:15:00,409.0


In [4]:
test.head()

,id,metropolitan_area_code,date
0,0,3,2022-01-01 00:45:00
1,1,3,2022-01-01 01:15:00
2,2,3,2022-01-01 01:30:00
3,3,3,2022-01-01 02:00:00
4,4,3,2022-01-01 02:15:00


- Is Null sum

In [5]:
train.isna().sum().any(), test.isna().sum().any()

(False, False)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717414 entries, 0 to 717413
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   metropolitan_area_code  717414 non-null  int64  
 1   date                    717414 non-null  object 
 2   energy_consumption      717414 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 16.4+ MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60870 entries, 0 to 60869
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      60870 non-null  int64 
 1   metropolitan_area_code  60870 non-null  int64 
 2   date                    60870 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [8]:
print(len(train['metropolitan_area_code'].unique()))

train['metropolitan_area_code'].value_counts()

17


2     78477
16    65256
3     62417
11    59587
0     55893
4     54914
14    54364
9     52229
1     49754
13    38024
15    29976
12    26392
8     24992
6     23541
10    18971
7     18654
5      3973
Name: metropolitan_area_code, dtype: int64

In [9]:
print(len(test['metropolitan_area_code'].unique()))

test['metropolitan_area_code'].value_counts()

17


2     6858
7     4696
10    4644
8     4581
0     4553
3     4533
4     4433
6     4271
5     3559
13    2974
9     2742
14    2741
11    2732
1     2279
16    2262
15    2244
12     768
Name: metropolitan_area_code, dtype: int64

# Preprocessing Data

In [10]:
train["date"] = pd.to_datetime(train["date"])
test["date"] = pd.to_datetime(test["date"])

train.head()

,metropolitan_area_code,date,energy_consumption
0,3,2017-09-11 02:15:00,477.0
1,3,2017-09-11 02:30:00,454.0
2,3,2017-09-11 03:45:00,398.0
3,3,2017-09-11 04:45:00,398.0
4,3,2017-09-11 05:15:00,409.0


# Feature Engineering

In [11]:
!pip install fast_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 362.4 kB/s eta 0:00:00


In [12]:
from fast_ml.feature_engineering import FeatureEngineering_DateTime
dt_fe = FeatureEngineering_DateTime()

dt_fe.fit(train, datetime_variables=['date'])
train = dt_fe.transform(train)

train.head(3)

/opt/conda/lib/python3.7/site-packages/fast_ml/feature_engineering.py:382: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[pfx+f] = getattr(df[var].dt, f)


,metropolitan_area_code,date,energy_consumption,date:year,date:quarter,date:month,date:day,date:day_of_week,date:day_of_year,date:weekofyear,...,date:is_quarter_end,date:is_quarter_start,date:is_year_end,date:is_year_start,date:time,date:hour,date:minute,date:second,date:is_weekend,date:day_part
0,3,2017-09-11 02:15:00,477.0,2017,3,9,11,0,254,37,...,False,False,False,False,02:15:00,2,15,0,0,midnight
1,3,2017-09-11 02:30:00,454.0,2017,3,9,11,0,254,37,...,False,False,False,False,02:30:00,2,30,0,0,midnight
2,3,2017-09-11 03:45:00,398.0,2017,3,9,11,0,254,37,...,False,False,False,False,03:45:00,3,45,0,0,midnight


#### FE : Test Data

In [13]:
dt_fe.fit(test, datetime_variables=['date'])
test = dt_fe.transform(test)

test.head(3)

/opt/conda/lib/python3.7/site-packages/fast_ml/feature_engineering.py:382: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[pfx+f] = getattr(df[var].dt, f)


,id,metropolitan_area_code,date,date:year,date:quarter,date:month,date:day,date:day_of_week,date:day_of_year,date:weekofyear,...,date:is_quarter_end,date:is_quarter_start,date:is_year_end,date:is_year_start,date:time,date:hour,date:minute,date:second,date:is_weekend,date:day_part
0,0,3,2022-01-01 00:45:00,2022,1,1,1,5,1,52,...,False,True,False,True,00:45:00,0,45,0,1,None
1,1,3,2022-01-01 01:15:00,2022,1,1,1,5,1,52,...,False,True,False,True,01:15:00,1,15,0,1,midnight
2,2,3,2022-01-01 01:30:00,2022,1,1,1,5,1,52,...,False,True,False,True,01:30:00,1,30,0,1,midnight


### Drop unique columns

In [14]:
nunique_train=train.nunique().reset_index()
remove_col=nunique_train[(nunique_train[0]==len(train)) | (nunique_train[0]==0) | (nunique_train[0]==1) ]['index'].tolist()
remove_col

['date:second']

In [15]:
print("Before :", train.shape)
train = train.drop(remove_col,axis=1)
print("After :", train.shape)

train.head(3)

Before : (717414, 22)
After : (717414, 21)


,metropolitan_area_code,date,energy_consumption,date:year,date:quarter,date:month,date:day,date:day_of_week,date:day_of_year,date:weekofyear,...,date:is_month_start,date:is_quarter_end,date:is_quarter_start,date:is_year_end,date:is_year_start,date:time,date:hour,date:minute,date:is_weekend,date:day_part
0,3,2017-09-11 02:15:00,477.0,2017,3,9,11,0,254,37,...,False,False,False,False,False,02:15:00,2,15,0,midnight
1,3,2017-09-11 02:30:00,454.0,2017,3,9,11,0,254,37,...,False,False,False,False,False,02:30:00,2,30,0,midnight
2,3,2017-09-11 03:45:00,398.0,2017,3,9,11,0,254,37,...,False,False,False,False,False,03:45:00,3,45,0,midnight


In [16]:
print("Before :", test.shape)
test = test.drop(remove_col,axis=1)
print("After :", test.shape)

test.head(3)

Before : (60870, 22)
After : (60870, 21)


,id,metropolitan_area_code,date,date:year,date:quarter,date:month,date:day,date:day_of_week,date:day_of_year,date:weekofyear,...,date:is_month_start,date:is_quarter_end,date:is_quarter_start,date:is_year_end,date:is_year_start,date:time,date:hour,date:minute,date:is_weekend,date:day_part
0,0,3,2022-01-01 00:45:00,2022,1,1,1,5,1,52,...,True,False,True,False,True,00:45:00,0,45,1,None
1,1,3,2022-01-01 01:15:00,2022,1,1,1,5,1,52,...,True,False,True,False,True,01:15:00,1,15,1,midnight
2,2,3,2022-01-01 01:30:00,2022,1,1,1,5,1,52,...,True,False,True,False,True,01:30:00,1,30,1,midnight


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717414 entries, 0 to 717413
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   metropolitan_area_code  717414 non-null  int64         
 1   date                    717414 non-null  datetime64[ns]
 2   energy_consumption      717414 non-null  float64       
 3   date:year               717414 non-null  int64         
 4   date:quarter            717414 non-null  int64         
 5   date:month              717414 non-null  int64         
 6   date:day                717414 non-null  int64         
 7   date:day_of_week        717414 non-null  int64         
 8   date:day_of_year        717414 non-null  int64         
 9   date:weekofyear         717414 non-null  int64         
 10  date:is_month_end       717414 non-null  bool          
 11  date:is_month_start     717414 non-null  bool          
 12  date:is_quarter_end     717414

In [18]:
import re

train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

train.head(3)

,metropolitan_area_code,date,energy_consumption,dateyear,datequarter,datemonth,dateday,dateday_of_week,dateday_of_year,dateweekofyear,...,dateis_month_start,dateis_quarter_end,dateis_quarter_start,dateis_year_end,dateis_year_start,datetime,datehour,dateminute,dateis_weekend,dateday_part
0,3,2017-09-11 02:15:00,477.0,2017,3,9,11,0,254,37,...,False,False,False,False,False,02:15:00,2,15,0,midnight
1,3,2017-09-11 02:30:00,454.0,2017,3,9,11,0,254,37,...,False,False,False,False,False,02:30:00,2,30,0,midnight
2,3,2017-09-11 03:45:00,398.0,2017,3,9,11,0,254,37,...,False,False,False,False,False,03:45:00,3,45,0,midnight


In [19]:
bool_cols = [col for col in train.columns if train[col].dtypes == 'bool']

for col in bool_cols:
    train[col] = train[col].map({1:True, 0:False})
    test[col] = test[col].map({1:True, 0:False})

In [20]:
test.isna().sum()

id                            0
metropolitan_area_code        0
date                          0
dateyear                      0
datequarter                   0
datemonth                     0
dateday                       0
dateday_of_week               0
dateday_of_year               0
dateweekofyear                0
dateis_month_end          60870
dateis_month_start        60870
dateis_quarter_end        60870
dateis_quarter_start      60870
dateis_year_end           60870
dateis_year_start         60870
datetime                      0
datehour                      0
dateminute                    0
dateis_weekend                0
dateday_part               2505
dtype: int64

In [21]:
train.isna().sum()

metropolitan_area_code         0
date                           0
energy_consumption             0
dateyear                       0
datequarter                    0
datemonth                      0
dateday                        0
dateday_of_week                0
dateday_of_year                0
dateweekofyear                 0
dateis_month_end          717414
dateis_month_start        717414
dateis_quarter_end        717414
dateis_quarter_start      717414
dateis_year_end           717414
dateis_year_start         717414
datetime                       0
datehour                       0
dateminute                     0
dateis_weekend                 0
dateday_part               30234
dtype: int64

In [22]:
drop_cols = ['dateis_month_end','dateis_month_start','dateis_quarter_end','dateis_quarter_start',
            'dateis_year_end','dateis_year_start','dateday_part','datetime','date']

train = train.drop(drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)

train.shape, test.shape

((717414, 12), (60870, 12))

In [23]:
test = test.drop(['id'],axis=1)

#train.shape, test.shape

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717414 entries, 0 to 717413
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   metropolitan_area_code  717414 non-null  int64  
 1   energy_consumption      717414 non-null  float64
 2   dateyear                717414 non-null  int64  
 3   datequarter             717414 non-null  int64  
 4   datemonth               717414 non-null  int64  
 5   dateday                 717414 non-null  int64  
 6   dateday_of_week         717414 non-null  int64  
 7   dateday_of_year         717414 non-null  int64  
 8   dateweekofyear          717414 non-null  int64  
 9   datehour                717414 non-null  int64  
 10  dateminute              717414 non-null  int64  
 11  dateis_weekend          717414 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 65.7 MB


In [25]:
test.head(5)

,metropolitan_area_code,dateyear,datequarter,datemonth,dateday,dateday_of_week,dateday_of_year,dateweekofyear,datehour,dateminute,dateis_weekend
0,3,2022,1,1,1,5,1,52,0,45,1
1,3,2022,1,1,1,5,1,52,1,15,1
2,3,2022,1,1,1,5,1,52,1,30,1
3,3,2022,1,1,1,5,1,52,2,0,1
4,3,2022,1,1,1,5,1,52,2,15,1


# Splitting Data

In [26]:
X = train.copy()
y = X.pop('energy_consumption')

In [27]:
# Save data
train.to_csv("rte_train_preprocessing.csv", index=False)
test.to_csv("rte_test_preprocessing.csv", index=False)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.0005, random_state=42, shuffle=True)

X_train.shape, X_test.shape, y_test.shape, y_train.shape

((717055, 11), (359, 11), (359,), (717055,))

# Modeling : EvalML

In [29]:
!python3 -m pip install -q evalml==0.28.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
tfx-bsl 1.8.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.48.0 which is incompatible.
tfx-bsl 1.8.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tfx-bsl 1.8.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<3,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
tensorflow-transform 1.8.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-transform 1.8.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<2.9,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.7 which is inco

In [30]:
from evalml.automl import AutoMLSearch

# run model
model_evalml = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='regression', max_time=3200) # 60 minutes
model_evalml.search()

Generating pipelines to search over...


8 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Will stop searching for new pipelines after 3200 seconds.

Allowed model families: xgboost, random_forest, linear_model, extra_trees, lightgbm, catboost, decision_tree



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline
Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean R2: -0.000

*****************************
* Evaluating Batch Number 1 *
*****************************

Linear Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.433
Decision Tree Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.976
Random Forest Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.978
LightGBM Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.967


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.625e+10, tolerance: 2.319e+07



Elastic Net Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.431
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.994
Extra Trees Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.960
CatBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.430

*****************************
* Evaluating Batch Number 2 *
*****************************

XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.998
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.998
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.997
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.997
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross 

In [31]:
# check leaderboard
model_evalml.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,13,XGBoost Regressor w/ Imputer,13,0.998621,NaN,0.998621,7.204062e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,15,Random Forest Regressor w/ Imputer,15,0.996947,NaN,0.996947,7.191985e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
10,2,Decision Tree Regressor w/ Imputer,2,0.976055,NaN,0.976055,7.041270e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
13,4,LightGBM Regressor w/ Imputer,4,0.967309,NaN,0.967309,6.978179e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
16,7,Extra Trees Regressor w/ Imputer,7,0.960471,NaN,0.960471,6.928854e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
20,1,Linear Regressor w/ Imputer + Standard Scaler,1,0.433413,NaN,0.433413,3.126703e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
21,5,Elastic Net Regressor w/ Imputer + Standard Sc...,5,0.431225,NaN,0.431225,3.110916e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
22,8,CatBoost Regressor w/ Imputer,8,0.429633,NaN,0.429633,3.099435e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
23,0,Mean Baseline Regression Pipeline,0,-0.000014,NaN,-0.000014,0.000000e+00,False,{'Baseline Regressor': {'strategy': 'mean'}}


In [32]:
import math
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluates_model(y_test, y_pred):
    print("*"*12, "Evaluations", "*"*12, '\n')
    
    print("MAE model :", mean_absolute_error(y_test, y_pred))
    
    print("MSE model :", mean_squared_error(y_test, y_pred))
    
    print("R2_Score model :", r2_score(y_test, y_pred))
    
    mse_1 = np.square(np.subtract(y_test,y_pred)).mean() 
    
    print("RMSE model :", math.sqrt(mse_1))

In [33]:
model_evalml.best_pipeline

pipeline = RegressionPipeline(component_graph={'Imputer': ['Imputer'], 'XGBoost Regressor': ['XGBoost Regressor', 'Imputer.x']}, parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'XGBoost Regressor':{'eta': 0.05671392060446587, 'max_depth': 13, 'min_child_weight': 4.450973669432, 'n_estimators': 487, 'n_jobs': -1}}, random_seed=0)

In [34]:
pred = model_evalml.best_pipeline.predict(X_test)
evaluates_model(y_test, pred)

************ Evaluations ************ 

MAE model : 13.526964086676042
MSE model : 588.5816316468857
R2_Score model : 0.9994887181714366
RMSE model : 291.5135192871094


# Test Prediction

In [35]:
pred = model_evalml.best_pipeline.predict(test)
len(pred), test.shape

(60870, (60870, 11))

# Submission

In [36]:
sub = pd.read_csv("../input/rte-forecast-energy-consumption-in-french-areas/sample_submission.csv")
sub.shape

(60870, 2)

In [37]:
!pip install pandas -U
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
woodwork 0.4.2 requires pandas<1.2.5,>=1.2.0, but you have pandas 1.3.5 which is incompatible.
tfx-bsl 1.8.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.48.0 which is incompatible.
tfx-bsl 1.8.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tfx-bsl 1.8.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<3,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
tensorflow-transform 1.8.0 re

In [38]:
sub['energy_consumption'] = pred
sub.to_csv('submission.csv', index=False)

In [39]:
sub

,id,energy_consumption
0,0,777.613831
1,1,797.869446
2,2,790.243530
3,3,780.007751
4,4,793.317627
...,...,...
60865,60865,357.377563
60866,60866,347.469269
60867,60867,395.113251
60868,60868,390.989990
